In [ ]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [ ]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(2, 1, (3,1), activation=['linear', 'linear'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
def get_train_test(url, split_percent=0.8):
    df = read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [ ]:
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))
    return X, Y

time_steps = 12
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

In [ ]:
model = create_RNN(hidden_units=3, dense_units=1, input_shape=(time_steps,1),
                   activation=['tanh', 'tanh'])
model.fit(trainX, trainY, epochs=15, batch_size=1, verbose=2)

Epoch 1/15
187/187 - 2s - 9ms/step - loss: 0.0068
Epoch 2/15
187/187 - 0s - 2ms/step - loss: 0.0055
Epoch 3/15
187/187 - 1s - 3ms/step - loss: 0.0050
Epoch 4/15
187/187 - 0s - 2ms/step - loss: 0.0047
Epoch 5/15
187/187 - 1s - 3ms/step - loss: 0.0045
Epoch 6/15
187/187 - 1s - 3ms/step - loss: 0.0043
Epoch 7/15
187/187 - 1s - 3ms/step - loss: 0.0041
Epoch 8/15
187/187 - 0s - 3ms/step - loss: 0.0040
Epoch 9/15
187/187 - 1s - 3ms/step - loss: 0.0040
Epoch 10/15
187/187 - 1s - 7ms/step - loss: 0.0038
Epoch 11/15
187/187 - 1s - 4ms/step - loss: 0.0038
Epoch 12/15
187/187 - 1s - 4ms/step - loss: 0.0038
Epoch 13/15
187/187 - 1s - 5ms/step - loss: 0.0037
Epoch 14/15
187/187 - 1s - 4ms/step - loss: 0.0036
Epoch 15/15
187/187 - 1s - 3ms/step - loss: 0.0035


In [ ]:
def print_error(trainY, testY, train_predict, test_predict):
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    # Print RMSE
    print('Train RMSE: %.3f RMSE' % (train_rmse))
    print('Test RMSE: %.3f RMSE' % (test_rmse))

# make predictions
train_predict = model.predict(trainX)
test_predict = model.predict(testX)
# Mean square error
print_error(trainY, testY, train_predict, test_predict)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Train RMSE: 0.059 RMSE
Test RMSE: 0.095 RMSE
